In [1]:
import os
import pprint
import pandas as pd
import json

import requests

from tastytrade import Credentials
from tastytrade.session import (
    SessionHandler,
)
from tastytrade.intruments import request_options_chains

from injector import Injector, singleton

from tastytrade.utilties import logger

TEST = True
ENV = "Test"

In [ ]:
def configure(binder):
    binder.bind(Credentials, to=Credentials(env=ENV), scope=singleton)


injector = Injector([configure])
session = injector.get(SessionHandler)

# session.create_session()
session.get_api_quote_token()

In [ ]:
print("\n", pprint.pformat(vars(session)))

if TEST:
    session.create_session(**vars(Credentials(env=ENV)))

In [ ]:
new_response = request_options_chains(session, "SPX")

In [ ]:
options_chains_df = pd.DataFrame(new_response.json()["data"]["items"])

is_tradeble = options_chains_df["active"] & ~options_chains_df["is-closing-only"]
options_chains_df.loc[is_tradeble].head(3)

In [ ]:
columns = [
    "underlying-symbol",
    "option-type",
    "strike-price",
    "streamer-symbol",
    "symbol",
]

rows = (
    (options_chains_df["settlement-type"] == options_chains_df["settlement-type"].max())
    & (options_chains_df["days-to-expiration"] == options_chains_df["days-to-expiration"].min())
    & (options_chains_df["active"] == 1)
    & ~options_chains_df["is-closing-only"]
)

options_chains_df.loc[rows, columns].sample(5)

In [ ]:
options_chains_df.loc[rows, columns]

In [ ]:
session.close_session()

# WebSocket Implementation

I will be using [Websockets](https://websockets.readthedocs.io/en/stable/), not to be confused with `websocket` which is a low-level library.

* [Documentation](https://websockets.readthedocs.io/en/stable/) on readthedocs.io includes "Getting Started" and "How-To" guides
* [Example](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) using Threading - credit: [LordKaT](https://github.com/LordKaT)
 
According to the documentation, the package is built on `AsyncIO`. Here is a list of supported features: [features support matrix](https://websockets.readthedocs.io/en/stable/reference/features.html).  You'll notice the most exapnsive coverage comes with `AsyncIO`. 

Since I haven't yet had a good use case for building an application on this technology, having only touched tradiotnal Threads, I'll start off with `AsyncIO` and see how far I get.

In [9]:
import threading, time, websocket

# Based off WebSocket example found here: https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py